# First Step

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Store Part I results into DataFrame

In [17]:
data_df = pd.read_csv("../WeatherPy/raw_data/weather_cities_data.csv")
data_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Puerto Ayora,-0.7393,-90.3518,80.56,80,63,9.98,EC,1634668091
1,Jamestown,42.0970,-79.2353,67.30,50,1,3.00,US,1634668128
2,Busselton,-33.6500,115.3333,54.09,74,100,23.47,AU,1634667415
3,Yellowknife,62.4560,-114.3525,45.41,76,40,8.05,CA,1634667382
4,Ushuaia,-54.8000,-68.3000,58.66,47,40,21.85,AR,1634668066


# Humidity Heatmap

In [11]:
# Configure gmaps.
gmaps.configure(api_key=g_key)

In [26]:
# Use the Lat and Lng as locations and Humidity as the weight.
# Add Heatmap layer to map.


locations = data_df[["Lat","Lng"]]


humidity = data_df["Humidity"]


max_humidity = data_df["Humidity"].max()

In [30]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=max_humidity,point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

In [41]:
# Narrow down the cities to fit weather conditions.
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
# Drop any rows will null values.

good_weather_df = data_df.loc[
    
(data_df["Max Temp"] >= 70) & (data_df["Max Temp"] <= 80)  &    (data_df["Wind Speed"] <= 10) & (data_df["Cloudiness"]== 0) 
    
].dropna()

good_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
222,Caxias,38.7031,-9.2767,74.53,65,0,1.99,PT,1634668197
278,Avera,33.1940,-82.5271,75.94,33,0,2.06,US,1634668131
336,Camacha,33.0833,-16.3333,72.36,78,0,5.75,PT,1634668233
337,Sonoita,31.8500,-112.8333,74.53,44,0,9.86,MX,1634668233
421,Hambantota,6.1241,81.1185,77.34,100,0,2.30,LK,1634668255


# Hotel Map

In [51]:
hotel_df = good_weather_df 

hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
222,Caxias,38.7031,-9.2767,74.53,65,0,1.99,PT,1634668197,Solar Palmeiras
278,Avera,33.1940,-82.5271,75.94,33,0,2.06,US,1634668131,
336,Camacha,33.0833,-16.3333,72.36,78,0,5.75,PT,1634668233,Hotel Porto Santo & Spa
337,Sonoita,31.8500,-112.8333,74.53,44,0,9.86,MX,1634668233,Hotel Excelsior
421,Hambantota,6.1241,81.1185,77.34,100,0,2.30,LK,1634668255,The Peacock Beach Hotel Hambantota


In [52]:
hotel_df.count()

City          8
Lat           8
Lng           8
Max Temp      8
Humidity      8
Cloudiness    8
Wind Speed    8
Country       8
Date          8
Hotel Name    8
dtype: int64

In [44]:
# set up additional columns to hold information

hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
222,Caxias,38.7031,-9.2767,74.53,65,0,1.99,PT,1634668197,
278,Avera,33.1940,-82.5271,75.94,33,0,2.06,US,1634668131,
336,Camacha,33.0833,-16.3333,72.36,78,0,5.75,PT,1634668233,
337,Sonoita,31.8500,-112.8333,74.53,44,0,9.86,MX,1634668233,
421,Hambantota,6.1241,81.1185,77.34,100,0,2.30,LK,1634668255,


In [45]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.

In [46]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = { "radius": 5000,
          
            "keyword": "hotel",
          
            "type": "hotel",
          
            "key": g_key,
       
    
}





In [48]:
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
    
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    
    
    except (KeyError, IndexError):
        print("Data not found... skipping.")
        
        
    print("------------------------------")
    

Retrieving Results for Index 222: Caxias.
Closest hotel in Caxias is Solar Palmeiras.
------------------------------
Retrieving Results for Index 278: Avera.
Data not found... skipping.
------------------------------
Retrieving Results for Index 336: Camacha.
Closest hotel in Camacha is Hotel Porto Santo & Spa.
------------------------------
Retrieving Results for Index 337: Sonoita.
Closest hotel in Sonoita is Hotel Excelsior.
------------------------------
Retrieving Results for Index 421: Hambantota.
Closest hotel in Hambantota is The Peacock Beach Hotel Hambantota.
------------------------------
Retrieving Results for Index 455: Iraq.
Data not found... skipping.
------------------------------
Retrieving Results for Index 517: Bulnes.
Closest hotel in Bulnes is Casa de Adobe.
------------------------------
Retrieving Results for Index 527: Hernani.
Closest hotel in Hernani is Zenit San Sebastián.
------------------------------


In [53]:
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
222,Caxias,38.7031,-9.2767,74.53,65,0,1.99,PT,1634668197,Solar Palmeiras
278,Avera,33.1940,-82.5271,75.94,33,0,2.06,US,1634668131,
336,Camacha,33.0833,-16.3333,72.36,78,0,5.75,PT,1634668233,Hotel Porto Santo & Spa
337,Sonoita,31.8500,-112.8333,74.53,44,0,9.86,MX,1634668233,Hotel Excelsior
421,Hambantota,6.1241,81.1185,77.34,100,0,2.30,LK,1634668255,The Peacock Beach Hotel Hambantota
455,Iraq,33.0000,44.0000,75.11,34,0,6.69,IQ,1634668097,
517,Bulnes,-36.7423,-72.2985,77.76,25,0,4.18,CL,1634668287,Casa de Adobe
527,Hernani,43.2662,-1.9762,75.61,68,0,3.44,ES,1634668290,Zenit San Sebastián


In [66]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]





In [67]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))